Code to make h5 files of the templates in the case that they are not saved correctly during make_templates

Created December 7, 2022
Updated May 18, 2023

In [1]:
import pandas as pd #to work with csv and dataframe
import numpy as np #for array
from eqcorrscan import Tribe #import the ability to read .tgz files with templates
import h5py #for writing to h5
from glob import glob #for reading multiple files in a directory
from obspy import Trace, Stream #for reading in waveform data as a trace/stream

Matched-filter CPU is not compiled! Should be here: /home/jupyter_share/miniconda3/envs/seismo/lib/python3.8/site-packages/fast_matched_filter/lib/matched_filter_CPU.so


In [2]:
homedir = '/home/smocz/expand_redpy_new_files/templates/' #directory to with .tgz file
savedir = '/home/smocz/expand_redpy_new_files/h5/' #directory to save h5 file in
volc = 'St_Helens'
h5name = f'{volc}_template_waveforms.h5'

Saving all waveforms at a volcano to h5 from tgz

In [3]:
#pull in all tgz files in a directory for a volcano
path = '/home/smocz/expand_redpy_new_files/templates/' #path to .tgz file

wave_list = []
name_list = []

from glob import glob
for filename_path in glob(path+f'Volcano_{volc}_*.tgz'): #get name and path for all tgz files in path variable, filename is a str
    T = Tribe().read(filename_path) #read the .tgz file
    for t in T: #for each template (t) in the .tgz (T)
        print('--------------') #divider between templates
#         print(t) #print the template details
#         print(' ') #divider
#         print('Template Name', end=" ") #label
        print(t.name) #print the template name
#         t.st.plot() #show the obspy stream of the template
        wave_list.append(np.array(t.st[0].data)) #append stream data as an array
        name_list.append(t.name) #append names to list
#         break
#     break

wave_arr = np.array(wave_list)
print(wave_arr.shape) #should be (# of streams, length of streams) like (90, 3121) for 90 streams of length 3121

--------------
awstehzrpst0022
--------------
awstehzrpst0145
--------------
awstehzrpst0265
--------------
awstehzrpst0329
--------------
awstehzrpst0348
--------------
awstehzrpst0590
--------------
awstehzrpst0591
--------------
awstehzrpst0593
--------------
awstehzrpst0598
--------------
awstehzrpst0602
--------------
awstehzrpst0603
--------------
awstehzrpst0612
--------------
awstehzrpst0613
--------------
awstehzrpst0617
--------------
awstehzrpst0625
--------------
awstehzrpst0627
--------------
awstehzrpst0634
--------------
awstehzrpst0638
--------------
awstehzrpst0639
--------------
awstehzrpst0640
--------------
awstehzrpst0647
--------------
awstehzrpst0648
--------------
awstehzrpst0656
--------------
awstehzrpst0657
--------------
awstehzrpst0661
--------------
awstehzrpst0668
--------------
awstehzrpst2002
--------------
awstehzrpst2016
--------------
awstehzrpst2021
--------------
awstehzrpst2027
--------------
awstehzrpst2028
--------------
awstehzrpst2039
--------

--------------
edmehzrpst0000
--------------
edmehzrpst0001
--------------
edmehzrpst0002
--------------
edmehzrpst0003
--------------
edmehzrpst0004
--------------
edmehzrpst0005
--------------
edmehzrpst0006
--------------
edmehzrpst0007
--------------
edmehzrpst0008
--------------
edmehzrpst0009
--------------
edmehzrpst0010
--------------
edmehzrpst0011
--------------
edmehzrpst0012
--------------
edmehzrpst0013
--------------
edmehzrpst0014
--------------
edmehzrpst0015
--------------
edmehzrpst0016
--------------
edmehzrpst0017
--------------
edmehzrpst0018
--------------
edmehzrpst0019
--------------
edmehzrpst0020
--------------
edmehzrpst0021
--------------
edmehzrpst0022
--------------
edmehzrpst0023
--------------
edmehzrpst0024
--------------
edmehzrpst0025
--------------
edmehzrpst0026
--------------
edmehzrpst0027
--------------
edmehzrpst0028
--------------
edmehzrpst0029
--------------
edmehzrpst0030
--------------
edmehzrpst0031
--------------
edmehzrpst0033
----------

--------------
nedehzrpst0000
--------------
nedehzrpst0001
--------------
nedehzrpst0002
--------------
nedehzrpst0007
--------------
nedehzrpst0008
--------------
nedehzrpst0009
--------------
nedehzrpst0010
--------------
nedehzrpst0011
--------------
nedehzrpst0013
--------------
nedehzrpst0015
--------------
nedehzrpst0017
--------------
nedehzrpst0018
--------------
nedehzrpst0019
--------------
nedehzrpst0020
--------------
nedehzrpst0021
--------------
nedehzrpst0022
--------------
nedehzrpst0023
--------------
nedehzrpst0024
--------------
nedehzrpst0025
--------------
nedehzrpst0026
--------------
nedehzrpst0028
--------------
nedehzrpst0029
--------------
nedehzrpst0030
--------------
nedehzrpst0031
--------------
nedehzrpst0032
--------------
nedehzrpst0034
--------------
nedehzrpst0036
--------------
nedehzrpst0038
--------------
nedehzrpst0042
--------------
nedehzrpst0043
--------------
nedehzrpst0051
--------------
nedehzrpst0052
--------------
nedehzrpst0053
----------

--------------
rembhzrpst0022
--------------
rembhzrpst0265
--------------
rembhzrpst0329
--------------
rembhzrpst0348
--------------
rembhzrpst0384
--------------
rembhzrpst0427
--------------
rembhzrpst0485
--------------
rembhzrpst0562
--------------
rembhzrpst0564
--------------
rembhzrpst0593
--------------
rembhzrpst0662
--------------
rembhzrpst0664
--------------
rembhzrpst0673
--------------
rembhzrpst0678
--------------
rembhzrpst0680
--------------
rembhzrpst0681
--------------
rembhzrpst0682
--------------
rembhzrpst0683
--------------
rembhzrpst0684
--------------
rembhzrpst0685
--------------
rembhzrpst0686
--------------
rembhzrpst0687
--------------
rembhzrpst0690
--------------
rembhzrpst0691
--------------
rembhzrpst0692
--------------
rembhzrpst0693
--------------
rembhzrpst0694
--------------
rembhzrpst0696
--------------
rembhzrpst0697
--------------
rembhzrpst0699
--------------
rembhzrpst0700
--------------
rembhzrpst0701
--------------
rembhzrpst0702
----------

--------------
sepehzrpst0000
--------------
sepehzrpst0001
--------------
sepehzrpst0002
--------------
sepehzrpst0005
--------------
sepehzrpst0006
--------------
sepehzrpst0007
--------------
sepehzrpst0008
--------------
sepehzrpst0009
--------------
sepehzrpst0010
--------------
sepehzrpst0011
--------------
seprpst0012
--------------
sepehzrpst0013
--------------
sepehzrpst0015
--------------
sepehzrpst0016
--------------
sepehzrpst0018
--------------
sepehzrpst0019
--------------
seprpst0022
--------------
sepehzrpst0023
--------------
sepehzrpst0024
--------------
sepehzrpst0025
--------------
sepehzrpst0026
--------------
sepehzrpst0028
--------------
sepehzrpst0029
--------------
sepehzrpst0031
--------------
sepehzrpst0033
--------------
sepehzrpst0034
--------------
sepehzrpst0036
--------------
sepehzrpst0037
--------------
sepehzrpst0038
--------------
sepehzrpst0039
--------------
sepehzrpst0040
--------------
sepehzrpst0041
--------------
sepehzrpst0042
--------------
s

...

--------------
mi09bhzrpst0012
--------------
mi09bhzrpst0022
--------------
mi09bhzrpst0178
--------------
mi09bhzrpst0236
--------------
mi09bhzrpst0252
--------------
mi09bhzrpst0295
--------------
mi09bhzrpst0298
--------------
mi09bhzrpst0305
--------------
mi09bhzrpst0312
--------------
mi09bhzrpst0314
--------------
mi09bhzrpst0326
--------------
mi09bhzrpst0329
--------------
mi09bhzrpst0330
--------------
mi09bhzrpst0337
--------------
mi09bhzrpst0338
--------------
mi09bhzrpst0345
--------------
mi09bhzrpst0346
--------------
mi09bhzrpst0348
--------------
mi09bhzrpst0349
--------------
mi09bhzrpst0350
--------------
mi09bhzrpst0351
--------------
mi09bhzrpst0354
--------------
mi09bhzrpst0360
--------------
mi09bhzrpst0365
--------------
mi09bhzrpst0366
--------------
mi09bhzrpst0367
--------------
mi09bhzrpst0368
--------------
mi09bhzrpst0370
--------------
mi09bhzrpst0371
--------------
mi09bhzrpst0373
--------------
mi09bhzrpst0374
--------------
mi09bhzrpst0382
--------

--------------
mj09bhzrpst0012
--------------
mj09bhzrpst0022
--------------
mj09bhzrpst0178
--------------
mj09bhzrpst0236
--------------
mj09bhzrpst0252
--------------
mj09bhzrpst0295
--------------
mj09bhzrpst0298
--------------
mj09bhzrpst0305
--------------
mj09bhzrpst0312
--------------
mj09bhzrpst0314
--------------
mj09bhzrpst0326
--------------
mj09bhzrpst0329
--------------
mj09bhzrpst0330
--------------
mj09bhzrpst0333
--------------
mj09bhzrpst0337
--------------
mj09bhzrpst0338
--------------
mj09bhzrpst0345
--------------
mj09bhzrpst0346
--------------
mj09bhzrpst0348
--------------
mj09bhzrpst0349
--------------
mj09bhzrpst0350
--------------
mj09bhzrpst0351
--------------
mj09bhzrpst0354
--------------
mj09bhzrpst0360
--------------
mj09bhzrpst0365
--------------
mj09bhzrpst0367
--------------
mj09bhzrpst0368
--------------
mj09bhzrpst0370
--------------
mj09bhzrpst0371
--------------
mj09bhzrpst0373
--------------
mj09bhzrpst0374
--------------
mj09bhzrpst0382
--------

--------------
mk10bhzrpst0012
--------------
mk10bhzrpst0022
--------------
mk10bhzrpst0236
--------------
mk10bhzrpst0298
--------------
mk10bhzrpst0305
--------------
mk10bhzrpst0312
--------------
mk10bhzrpst0314
--------------
mk10bhzrpst0317
--------------
mk10bhzrpst0326
--------------
mk10bhzrpst0329
--------------
mk10bhzrpst0330
--------------
mk10bhzrpst0333
--------------
mk10bhzrpst0337
--------------
mk10bhzrpst0338
--------------
mk10bhzrpst0345
--------------
mk10bhzrpst0346
--------------
mk10bhzrpst0348
--------------
mk10bhzrpst0349
--------------
mk10bhzrpst0351
--------------
mk10bhzrpst0354
--------------
mk10bhzrpst0360
--------------
mk10bhzrpst0365
--------------
mk10bhzrpst0367
--------------
mk10bhzrpst0368
--------------
mk10bhzrpst0370
--------------
mk10bhzrpst0371
--------------
mk10bhzrpst0388
--------------
mk10bhzrpst0398
--------------
mk10bhzrpst0403
--------------
mk10bhzrpst0405
--------------
mk10bhzrpst0408
--------------
mk10bhzrpst0409
--------

--------------
mm04bhzrpst0012
--------------
mm04bhzrpst0022
--------------
mm04bhzrpst0178
--------------
mm04bhzrpst0236
--------------
mm04bhzrpst0298
--------------
mm04bhzrpst0305
--------------
mm04bhzrpst0310
--------------
mm04bhzrpst0312
--------------
mm04bhzrpst0314
--------------
mm04bhzrpst0317
--------------
mm04bhzrpst0326
--------------
mm04bhzrpst0329
--------------
mm04bhzrpst0330
--------------
mm04bhzrpst0337
--------------
mm04bhzrpst0338
--------------
mm04bhzrpst0345
--------------
mm04bhzrpst0346
--------------
mm04bhzrpst0348
--------------
mm04bhzrpst0349
--------------
mm04bhzrpst0350
--------------
mm04bhzrpst0351
--------------
mm04bhzrpst0354
--------------
mm04bhzrpst0355
--------------
mm04bhzrpst0360
--------------
mm04bhzrpst0365
--------------
mm04bhzrpst0368
--------------
mm04bhzrpst0369
--------------
mm04bhzrpst0370
--------------
mm04bhzrpst0373
--------------
mm04bhzrpst0374
--------------
mm04bhzrpst0376
--------------
mm04bhzrpst0381
--------

--------------
mo06bhzrpst2021
--------------
mo06bhzrpst2027
--------------
mo06bhzrpst2039
--------------
mo06bhzrpst2040
--------------
mo06bhzrpst2050
--------------
mo06bhzrpst2065
--------------
mo06bhzrpst2071
--------------
mo06bhzrpst2081
--------------
mo06bhzrpst2108
--------------
mo06bhzrpst2121
--------------
mo06bhzrpst2123
--------------
mo06bhzrpst2125
--------------
mo06bhzrpst2127
--------------
mo06bhzrpst2128
--------------
mo06bhzrpst2129
--------------
mo06bhzrpst2130
--------------
mo06bhzrpst2132
--------------
mo06bhzrpst2133
--------------
mo06bhzrpst2134
--------------
mo06bhzrpst2135
--------------
mo06bhzrpst2136
--------------
mo06bhzrpst2138
--------------
mo06bhzrpst2139
--------------
mo06bhzrpst2141
--------------
mo06bhzrpst2143
--------------
mo06bhzrpst2144
--------------
mo06bhzrpst2145
--------------
mo06bhzrpst2149
--------------
mo06bhzrpst2150
--------------
mo06bhzrpst2151
--------------
mo06bhzrpst2152
--------------
mo06bhzrpst2154
--------

In [5]:
with h5py.File(f"{savedir}{h5name}", "w") as f:
    f.create_dataset("waveforms", data=wave_arr)
    f.create_dataset("template_name", data=name_list)

Writing H5

In [ ]:
homedir = '/home/smocz/expand_redpy_new_files/templates/' #directory to with .tgz file
filename = 'Volcano_Rainier_Network_UW_Station_RCM_Channel_HHZ.tgz' #name of .tgz file
savedir = '/home/smocz/expand_redpy_new_files/h5/' #directory to save h5 file in
h5name = 'selected_waves.h5' #name of h5 file

In [ ]:
T = Tribe().read(f'{homedir}{filename}') #read the .tgz file

#create data
wave_list = [] #list of arrays of stream data
name_list = [] #list of template names

for t in T[:20]: #for each of the first 20 templates in the tgz file
    t_id = t.name #template name
    waveform = t.st #tempalte stream
    wave_list.append(np.array(t.st[0].data)) #append stream data as an array
    name_list.append(t.name) #append names to list
#     break
    
wave_arr = np.array(wave_list)
print(wave_arr.shape) #should be (# of streams, length of streams) like (90, 3121) for 90 streams of length 3121

In [ ]:
#add data to h5 file
with h5py.File(f"./{h5name}", "w") as f:
    f.create_dataset("waveforms", data=wave_arr)
    f.create_dataset("template_name", data=name_list)

Reading H5

In [ ]:
#load waveforms from h5 file
with h5py.File("./specufex_data.h5", "r") as f:
    waveforms = f["waveforms"][()]
    template_name = f["template_name"].asstr()[()]

print(f"{len(waveforms)} waveforms in file")

tdf = pd.DataFrame({"template_name":template_name,"waveform":list(waveforms)})
tdf.head()